In [13]:
from __future__ import annotations
import pandas as pd
import dataclasses
import typing

In [14]:
import seaborn
import pandas as pd
import numpy as np

iris_df = seaborn.load_dataset("iris")
print(iris_df.head(3))

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa


In [18]:
@dataclasses.dataclass
class IrisData0:
    __df: pd.DataFrame

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}(size={self.__df.shape[0]})'
    
    @property
    def sepal_length(self) -> pd.Series:
        return self.__df['sepal_length']

    @property
    def species(self) -> pd.Series:
        return self.__df['species']
    
    def species_mean_dataframe(self) -> pd.DataFrame:
        return self.__df.groupby('species').mean()

    def species_mean(self) -> SpeciesMean0:
        return SpeciesMean0.from_iris_data(self)

@dataclasses.dataclass
class SpeciesMean0:
    __species_av: pd.DataFrame

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}(num_species={self.__species_av.shape[0]})'\

    def __getitem__(self, species_name: str) -> pd.Series:
        return self.__species_av.loc[species_name]
    
    @property
    def all_species(self) -> typing.List[str]:
        return list(self.__species_av.index)

    @classmethod
    def from_iris_data(cls, iris_data: IrisData0) -> typing.Self:
        return cls(iris_data.species_mean_dataframe())

idata = IrisData0(iris_df)
print(idata.species.value_counts())
print(idata.species_mean_dataframe())
#print(idata.species_mean().all_species)
SpeciesMean0.from_iris_data(idata)
idata.species_mean()

species
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64
            sepal_length  sepal_width  petal_length  petal_width
species                                                         
setosa             5.006        3.428         1.462        0.246
versicolor         5.936        2.770         4.260        1.326
virginica          6.588        2.974         5.552        2.026


SpeciesMean0(num_species=3)

In [16]:
class ColNames:
    sepal_length = 'sepal_length'
    sepal_width = 'sepal_width'
    petal_length = 'petal_length'
    petal_width = 'petal_width'
    species = 'species'

    @classmethod
    def all(cls) -> typing.List[str]:
        return [cls.sepal_length, cls.sepal_width, cls.petal_length, cls.petal_width, cls.species]

@dataclasses.dataclass
class IrisData1:
    __df: pd.DataFrame
    
    @classmethod
    def from_dataframe(cls, df: pd.DataFrame) -> typing.Self:
        return cls(
            df = df[ColNames.all()]
        )
    
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}(size={self.__df.shape[0]})'
    
    def head(self, *args) -> pd.DataFrame:
        return self.__df.head(*args)
    
    @property
    def sepal_length(self) -> pd.Series:
        return self.__df['sepal_length']

    @property
    def species(self) -> pd.Series:
        return self.__df['species']

    @property
    def sepal_length(self) -> pd.Series:
        return self.__df[ColNames.sepal_length]
    
    @property
    def species(self) -> pd.Series:
        return self.__df[ColNames.species]
    
    def filter_by_species(self, species: str) -> typing.Self:
        return self.__class__(self.__df.query(f'species == "{species}"'))

id1 = IrisData1.from_dataframe(iris_df)
id1

TypeError: IrisData1.__init__() got an unexpected keyword argument 'df'

In [ ]:
IrisData1(iris_df)

IrisData1(size=150)

In [ ]:
id1.filter_by_species('setosa')

IrisData1(size=50)

In [ ]:
@dataclasses.dataclass
class IrisSummary:
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

@dataclasses.dataclass
class IrisData2:
    sepal_length: pd.Series
    sepal_width: pd.Series
    petal_length: pd.Series
    petal_width: pd.Series
    species: pd.Series
    
    @classmethod
    def from_dataframe(cls, df: pd.DataFrame) -> typing.Self:
        return cls(
            sepal_length = df['sepal_length'].astype(np.float64),
            sepal_width = df['sepal_width'].astype(np.float64),
            petal_length = df['petal_length'].astype(np.float64),
            petal_width = df['petal_width'].astype(np.float64),
            species = df['species'],
        )
    
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}(size={self.sepal_length.shape[0]})'
    
    def map(self, trans_func: typing.Callable[[pd.Series], pd.Series]) -> typing.Self:
        '''Used internally to handle transformations.'''
        return self.__class__(
            sepal_length = trans_func(self.sepal_length),
            sepal_width = trans_func(self.sepal_width),
            petal_length = trans_func(self.petal_length),
            petal_width = trans_func(self.petal_width),
            species = trans_func(self.species),
        )
    
    def group_by(self, groups: pd.Series) -> typing.Dict[typing.Hashable, typing.Self]:
        '''Create separate groups of elements.'''
        groupings = dict()
        for grp in groups.unique():
            groupings[grp] = self.__class__(self.map(lambda v: v[groups==grp]))
        return groupings

    def summarize(self, groups: pd.Series, summary_func: typing.Callable[[pd.Series], pd.Series]) -> IrisSummary:
        '''Apply summary functions across groups.'''
        summaries = dict()
        for grp, grouping in self.group_by(groups).items():
            summaries = 


        return IrisSummary(
            sepal_length = summary_func(self.sepal_length),
            sepal_width = summary_func(self.sepal_width),
            petal_length = summary_func(self.petal_length),
            petal_width = summary_func(self.petal_width),
        )
    


    def filter(self, sel: pd.Series) -> typing.Self:
        return self._map(lambda v: v[sel])
    
    def filter_by_species(self, species: str) -> typing.Self:
        return self.filter(self.species==species)

    def group_by_species(self) -> typing.Dict[str, typing.Self]:
        groups = dict()
        for species in self.species.unique():
            groups[species] = self.filter(self.species==species)
        return groups
    
    def mean_by_species(self) -> SpeciesMean2:
        return SpeciesMean2(
            av_sepal_length = self.sepal_length.mean(),
            av_sepal_width = self.sepal_width.mean(),
            av_petal_length = self.petal_length.mean(),
            av_petal_width = self.petal_width.mean(),
        )
    
    def as_dataframe(self) -> pd.DataFrame:
        return pd.DataFrame.from_dict({
            'sepal_length': self.sepal_length,
            'sepal_width': self.sepal_width,
            'petal_length': self.petal_length,
            'petal_width': self.petal_width,
            'species': self.species,
        })
    
    def mean_by_species_2(self) -> SpeciesMean2:
        avdf = self.as_dataframe().groupby('species').mean()
        return SpeciesMean2(
            av_sepal_length = float(avdf['sepal_length']),
            av_sepal_width = float(avdf['sepal_width']),
            av_petal_length = float(avdf['petal_length']),
            av_petal_width = float(avdf['petal_width']),
        )

id2 = IrisData2.from_dataframe(iris_df)
id2

SyntaxError: invalid syntax (1630335577.py, line 50)

In [ ]:
id2.filter_by_species('setosa')

IrisData2(size=50)

In [ ]:
id2.mean_by_species()